DB 초기화 프로그램-PowerBI용 데이터 생성

In [2]:
!pip install pymysql > /dev/null

In [3]:
from google.colab import files
uploaded = files.upload()
filename = list(uploaded.keys())[0]

Saving mysql.json to mysql.json


In [1]:
import pandas as pd
import json

In [7]:
with open('mysql.json', 'r') as file:
    config_str = file.read()
config = json.loads(config_str)

In [8]:
import pymysql
conn = pymysql.connect(
    host = config['host'],
    user = config['user'],
    password = config['password'],
    database = config['database'],
    port = config['port']
)

In [9]:
conn = pymysql.connect(**config)

In [5]:
import random
random.seed(2021)

## products table 생성

In [16]:
# products 테이블 생성 sql
sql = """
    CREATE TABLE if NOT exists products(
        pid INT PRIMARY KEY AUTO_INCREMENT,
        pname VARCHAR(40) NOT NULL,
        pprice INT UNSIGNED NOT NULL,
        pcategory VARCHAR(20) NOT NULL,
        pcost INT UNSIGNED NOT NULL
    ) AUTO_INCREMENT=11;
"""

In [17]:
cur = conn.cursor()
cur.execute(sql)

0

In [18]:
camping_goods = ['텐트','타프','천막','침낭','매트','랜턴','취사용품',
                 '아이스박스','보조배터리','워터저그']
car_goods = ['블랙박스','내비게이션','하이패스','후방카메라','자동차TV',
             '카오디오','핸즈프리','헤드유닛','방진매트','윤활방청제']
health_goods = ['러닝머신','웨이트기구','복근운동기구','벨트','거꾸리',
                '훌라후프','트램펄린','스텝퍼','로잉머신','헬스사이클']
category_names = ['캠핑용품','자동차용품','헬스용품']

In [19]:
for item in camping_goods:
    price = random.randint(100,999) * 100
    rate = random.randint(85, 98)
    cost = int(round(price * rate / 1000) * 10)
    print(item, price, '캠핑', cost)

텐트 32600 캠핑 29990
타프 58400 캠핑 51980
천막 23900 캠핑 21270
침낭 10500 캠핑 8920
매트 64900 캠핑 62950
랜턴 33400 캠핑 29390
취사용품 87300 캠핑 75950
아이스박스 95000 캠핑 92150
보조배터리 35200 캠핑 32030
워터저그 53600 캠핑 50380


In [20]:

sql = "insert into products(pname,pprice,pcategory,pcost) values(%s,%s,%s,%s);"
for i, item_list in enumerate([camping_goods, car_goods, health_goods]):
    for item in item_list:
        price = random.randint(100,999) * 100
        rate = random.randint(85, 98)
        cost = int(round(price * rate / 1000) * 10)
        cur.execute(sql, (item, price, category_names[i], cost))
conn.commit()

In [21]:
# spid를 외래키(foreign key)로 설정
sql = """
    CREATE TABLE if NOT EXISTS sales(
        sid INT PRIMARY KEY AUTO_INCREMENT,
        sdate DATE NOT NULL,
        scompany VARCHAR(20) NOT NULL,
        spid INT NOT NULL,
        sunit INT NOT NULL,
        FOREIGN KEY(spid) REFERENCES products(pid)
    ) AUTO_INCREMENT=101;
"""

In [22]:
cur = conn.cursor()
cur.execute(sql)

0

In [23]:
calendar = {1:31,2:28,3:31,4:30,5:31,6:30,7:31,8:31,9:30,10:31,11:30,12:31}
company_list = ['멀티캠퍼스','한경아카데미','표준협회','생산성본부','테크브루']

for i in range(10):
    date = f'2020-1-{random.randint(1,31)}'
    company = company_list[random.randint(0,4)]
    pid = random.randint(11,40)
    unit = random.randint(1,20)
    print(date,company,pid,unit)

2020-1-24 표준협회 28 2
2020-1-18 표준협회 40 1
2020-1-18 테크브루 34 14
2020-1-6 멀티캠퍼스 12 1
2020-1-14 표준협회 12 20
2020-1-13 테크브루 26 12
2020-1-18 한경아카데미 19 20
2020-1-31 테크브루 19 11
2020-1-6 생산성본부 29 12
2020-1-17 생산성본부 16 18


In [24]:
sql = "insert into sales values(default,%s,%s,%s,%s);"
for year in range(2015,2021):
    for month, day in calendar.items():
        for i in range(20):
            date = f'{year}-{month}-{random.randint(1,day)}'
            company = company_list[random.randint(0,4)]
            pid = random.randint(11,40)
            unit = random.randint(1,20)
            cur.execute(sql, (date, company, pid, unit))
    conn.commit()

In [25]:
sql = """
    CREATE VIEW sales_book as
        SELECT sid, sdate, scompany, pid, pname, pcategory, pprice, sunit, pcost,
            pprice*sunit AS revenue, (pprice-pcost)*sunit AS profit
        FROM sales
        JOIN products
        ON sales.spid = products.pid
        ORDER BY sdate;
"""

In [26]:
cur = conn.cursor()
cur.execute(sql)

0